## Bayesian Exploration of beam emittance at LCLS-II

In [5]:
from scripts.evaluate_function.screen_image import measure_beamsize, measure_background
from scripts.characterize_emittance import characterize_emittance

In [6]:
# optionally add scripts location to path
if True:
    import sys
    sys.path.append("../../../")
    sys.path.append("../")

import xopt
print(xopt.__version__)


# set up data saving locations
#data_dir = "/home/physics3/ml_tuning/20230729_LCLS_Injector"

#run_name = "optimize_1"
#run_dir = f"{data_dir}/{run_name}"
#import os
#if not os.path.exists(run_dir):
#    os.mkdir(run_dir)

2.0a1+19.g208c72e


In [7]:
from utils import VARIABLE_RANGES, SCAN_VARIABLE, \
    MEASUREMENT_OPTIONS, IMAGE_CONSTRAINTS, SCREEN_NAME, BEAM_ENERGY, QUAD_LENGTH, \
    PV_TO_INTEGRATED_GRADIENT, DRIFT_LENGTH

## (Optional) Measure background image

In [ ]:
MEASURE_BACKGROUND = True
if MEASURE_BACKGROUND:
    measure_background(SCREEN_NAME)
    BACKGROUND_FILE = f"{SCREEN_NAME}_background.npy".replace(":","_")
else:
    BACKGROUND_FILE = None

In [ ]:
# verify background image
import numpy as np
import matplotlib.pyplot as plt
plt.imshow(np.load(BACKGROUND_FILE))

## Define measurement parameters

In [ ]:
from xopt import VOCS
import numpy as np
from emitopt.utils import get_quad_strength_conversion_factor


EXPLORATION_VARIABLES = ["QUAD:GUNB:212:1:BCTRL","QUAD:GUNB:212:2:BCTRL",
                         "SOLN:GUNB:212:BCTRL","SOLN:GUNB:823:BCTRL"]


# create conversion factor from PV value to geometric focusing strength
integrated_gradient_to_geometric_focusing_strength = get_quad_strength_conversion_factor(
        BEAM_ENERGY, QUAD_LENGTH
    )
QUAD_STRENGTH_SCALE =  PV_TO_INTEGRATED_GRADIENT * \
           integrated_gradient_to_geometric_focusing_strength

ROI = None
SCAN_NUMBER = 0

MEASUREMENT_OPTIONS["background"] = BACKGROUND_FILE


# define function to measure the total size on OTR4
def eval_beamsize(input_dict):
    results = measure_beamsize(input_dict)
    results["S_x_mm"] = results["Sx"] * 1e3
    results["S_y_mm"] = results["Sy"] * 1e3

    #add total beam size
    results["total_size"] = np.sqrt(results["Sx"]**2 + results["Sy"]**2)
    return results

def eval_emittance(inputs: dict):
    global SCAN_NUMBER

    emit_vocs = VOCS(
        variables = {SCAN_VARIABLE: VARIABLE_RANGES[SCAN_VARIABLE]},
        constants = inputs,
        observables = ["total_size"],
        constraints = IMAGE_CONSTRAINTS
    )

    print(f"running emittance characterization scan number {SCAN_NUMBER}")
    results = characterize_emittance(
        emit_vocs,
        eval_beamsize,
        quad_length=QUAD_LENGTH,
        drift_length=DRIFT_LENGTH,
        beam_energy=BEAM_ENERGY,
        quad_strength_scale_factor=QUAD_STRENGTH_SCALE,
        quad_strength_key=SCAN_VARIABLE,
        rms_x_key="S_x_mm",
        rms_y_key="S_y_mm",
        n_initial=3,
        dump_file=f"data/scan_{SCAN_NUMBER}.yml"
    )

    SCAN_NUMBER += 1
    return results

## Set up exploration

In [ ]:
from xopt.evaluator import Evaluator
from xopt.generators import BayesianExplorationGenerator
from xopt import Xopt

vocs = VOCS(
        variables = {ele: VARIABLE_RANGES[ele] for ele in EXPLORATION_VARIABLES},
        constants = MEASUREMENT_OPTIONS,
        observables = ["x_emittance_median"],
    )

evaluator = Evaluator(function=eval_emittance)
generator = BayesianExplorationGenerator(vocs=vocs)

# reset SCAN_NUMBER
SCAN_NUMBER = 0

X = Xopt(vocs=vocs, generator=generator, evaluator=evaluator)
X

In [ ]:
## Evaluate initial points
initial_points = []
X.evaluate_data(initial_points)

In [ ]:
## run exploration
n_steps = 10
for i in range(n_steps):
    X.step()